<a href="https://colab.research.google.com/github/nicolegg/Torch_Colab/blob/master/TorchMatriz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

operaciones matemáticas básicas con PyTorch. PyTorch es un marco computacional para construir modelos de aprendizaje automático profundo. Aquí, sin embargo, usaremos PyTorch para las matemáticas de la escuela secundaria primero, luego para una tarea de regresión simple y finalmente para limpiar una imagen con objetos periodicos "atómicos".

Importar PyTorch:

In [2]:
import torch 
print ("importado")

importado


Definir algunos tensonres

In [10]:
# Crear un tensor de 3 x 3 que contenga número aleatorios
a = torch.rand(3, 3)
# Crear un tensor de 3 x 3 que contenga todos
b = torch.ones(3, 3)
# Crear un tendos de 3 x 3 de identidad
c = torch.eye(3, 3)
print ("Creado")

Creado


Imprimir los tensores que acabamos de definir

In [11]:
print(a, type(a))
print(b, type(b))
print(c, type(c))

tensor([[0.8640, 0.0281, 0.7036],
        [0.8654, 0.4642, 0.3406],
        [0.1097, 0.5159, 0.8491]]) <class 'torch.Tensor'>
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) <class 'torch.Tensor'>
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]) <class 'torch.Tensor'>


Restar o sumar:

In [0]:
d = torch.sub(b, a)
e = torch.add(b,c)

Resultados de la suma y resta entre matrices

In [13]:
print(d, type(d))
print(e, type(e))

tensor([[0.1360, 0.9719, 0.2964],
        [0.1346, 0.5358, 0.6594],
        [0.8903, 0.4841, 0.1509]]) <class 'torch.Tensor'>
tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]]) <class 'torch.Tensor'>
